In [ ]:
from jinja2 import Template

# Step 1: 读取模板内容
with open('/home/tangziyi/RD-Agent/rdagent/components/coder/factor_coder/template_debug.jinjia2', 'r') as f:
    template_content = f.read()

# Step 2: 渲染模板
template = Template(template_content)
rendered_code = template.render(
    expression="SMA(CLOSE, 5, 1)", # "DELAY($high + $low / 2, 5)",
    factor_name="FACTOR_1"
    )

# Step 3: 打印渲染后的代码
print(rendered_code)
exec(rendered_code)


import pandas as pd
import numpy as np
from rdagent.components.coder.factor_coder.expr_parser import parse_expression, parse_symbol
from rdagent.components.coder.factor_coder.function_lib import (
    ABS, ADD, AND, CORR, COUNT, COVIANCE, DECAYLINEAR, DELAY, DELTA, DIVIDE, 
    EMA, EXP, FILTER, HIGHDAY, LOG, LOWDAY, MAX, MEAN, MEDIAN, MIN, MULTIPLY, 
    OR, POW, PROD, RANK, REGBETA, REGRESI, SEQUENCE, SIGN, SMA, SQRT, STD, 
    SUBTRACT, SUM, SUMAC, SUMIF, TS_MAX, TS_MIN, TS_RANK, TS_ZSCORE, WMA, ZSCORE
)


def calculate_factor():
    # 获取表达式
    expr = "SMA(CLOSE, 5, 1)"
    df = pd.read_hdf('/home/tangziyi/RD-Agent/rdagent/scenarios/qlib/experiment/factor_data_template/daily_pv_all.h5', key='data')
    # df = pd.read_hdf('/home/tangziyi/RD-Agent/git_ignore_folder/factor_implementation_source_data/daily_pv.h5', key='data')
    print(df.columns)
    expr = parse_symbol(expr, df.columns)
    # print(expr)
    expr = parse_expression(expr)

    for col in df.columns:
        expr = ex

In [10]:
from rdagent.components.coder.factor_coder.function_lib import *

for i in dir():
    if i.isupper():
        print(i, end=', ')

ABS, ADD, AND, CORR, COUNT, COVIANCE, DECAYLINEAR, DELAY, DELTA, DIVIDE, EMA, EXP, FILTER, HIGHDAY, LOG, LOWDAY, MAX, MEAN, MEDIAN, MIN, MULTIPLY, OR, POW, PROD, RANK, REGBETA, REGRESI, SEQUENCE, SIGN, SMA, SQRT, STD, SUBTRACT, SUM, SUMAC, SUMIF, TS_MAX, TS_MIN, TS_RANK, TS_ZSCORE, WMA, ZSCORE, 